In [1]:
# delete this cell if working on Pycharm
!pip install Bio
!pip install import-ipynb

In [2]:
!pip install wandb

In [14]:
print(len(tf.config.list_physical_devices('GPU')))

1


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

# so we can import utils notebook (delete if working on Pycharm), you might need to change it to your working directory path
%cd "/content/drive/MyDrive/ColabNotebooks"
import import_ipynb
import utils

/content/drive/MyDrive/ColabNotebooks
importing Jupyter notebook from utils.ipynb


In [4]:
###############################################################################
#                                                                             #
#              Parameters you can change, but don't have to                   #
#                                                                             #
###############################################################################


# number of ResNet blocks for the first ResNet and the kernel size.
RESNET_1_BLOCKS = 3 
RESNET_1_KERNEL_SIZE = 15
RESNET_1_KERNEL_NUM = 64


###############################################################################
#                                                                             #
#                        Parameters you need to choose                        #
#                                                                             #
###############################################################################


# number of ResNet blocks for the second ResNet, dilation list to repeat and the kernel size.

RESNET_2_BLOCKS = 1 
RESNET_2_KERNEL_SIZE = 1  # good start may be 3/5
RESNET_2_KERNEL_NUM = 1
DILATION = [1]

# percentage of dropout for the dropout layer
DROPOUT = 0.0 # good start may be 0.1-0.5

# number of epochs, Learning rate and Batch size
EPOCHS = 3
LR = 0.001 # good start may be 0.0001/0.001/0.01
BATCH = 128 # good start may be 32/64/128



In [6]:
def resnet_block(input_layer, kernel_size,kernel_num,dialation=1):
    bn1 = layers.BatchNormalization()(input_layer)
    conv1d_layer1 = layers.Conv1D(kernel_num,kernel_size,padding='same',activation='relu', dilation_rate=dialation)(bn1)
    bn2 = layers.BatchNormalization()(conv1d_layer1)
    conv1d_layer2 = layers.Conv1D(kernel_num,kernel_size,padding='same',activation='relu', dilation_rate=dialation)(bn2)
    return layers.Add()([input_layer,conv1d_layer2])

In [5]:
def resnet_1(input_layer,block_num=RESNET_1_BLOCKS, kernel_size=RESNET_1_KERNEL_SIZE, kernel_num=RESNET_1_KERNEL_NUM):  # TODO: implement this!
    """
    ResNet layer - input -> BatchNormalization -> Conv1D -> Relu -> BatchNormalization -> Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    last_layer_output = input_layer

    for i in range (block_num):
      last_layer_output = resnet_block(last_layer_output,kernel_size,kernel_num)

    return last_layer_output
 

    


In [7]:
def resnet_2(input_layer,block_num = RESNET_2_BLOCKS, kernel_size = RESNET_2_KERNEL_SIZE, kernel_num = RESNET_2_KERNEL_NUM, dial_lst=DILATION):  # TODO: implement this!
    """
    Dilated ResNet layer - input -> BatchNormalization -> dilated Conv1D -> Relu -> BatchNormalization -> dilated Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    last_layer_output = input_layer

    for i in range (block_num):
      for d in dial_lst:
        last_layer_output = resnet_block(last_layer_output,kernel_size,kernel_num,d)

    return last_layer_output

In [8]:
def build_network():
    """
    builds the neural network architecture as shown in the exercise.
    :return: a Keras Model
    """
    # input, shape (NB_MAX_LENGTH,FEATURE_NUM)
    input_layer = tf.keras.Input(shape=(utils.NB_MAX_LENGTH, utils.FEATURE_NUM))

    # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(RESNET_1_KERNEL_NUM, RESNET_1_KERNEL_SIZE, padding='same')(input_layer)

    # first ResNet -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    resnet_layer = resnet_1(conv1d_layer)

    # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(RESNET_2_KERNEL_NUM, RESNET_2_KERNEL_SIZE, padding="same")(resnet_layer)

    # second ResNet -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    resnet_layer = resnet_2(conv1d_layer)

    dp = layers.Dropout(DROPOUT)(resnet_layer)
    conv1d_layer = layers.Conv1D(RESNET_2_KERNEL_NUM // 2, RESNET_2_KERNEL_SIZE, padding="same", activation = 'elu')(dp)
    dense = layers.Dense(15)(conv1d_layer)

    return tf.keras.Model(input_layer, dense)
    


In [9]:
def plot_val_train_loss(history):
    """
    plots the train and validation loss of the model at each epoch, saves it in 'model_loss_history.png'
    :param history: history object (output of fit function)
    :return: None
    """
    ig, axes = plt.subplots(1, 1, figsize=(15,3))
    axes.plot(history.history['loss'], label='Training loss')
    axes.plot(history.history['val_loss'], label='Validation loss')
    axes.legend()
    axes.set_title("Train and Val MSE loss")

    plt.savefig("/content/drive/MyDrive/Ex4Files/model_loss_history")  # TODO: you can change the path here


In [10]:
def train(config=None):
  
  # you can load here your input and output data

  # X = numpy array of shape (1974,NB_MAX_LENGTH,FEATURE_NUM) of all the data input.
  # Y = numpy array of shape (1974,NB_MAX_LENGTH,OUTPUT_SIZE) of all the data labels.
  input = np.load("train_input.npy")
  labels = np.load("train_labels.npy")
  save_dir = "BestFits/"
  model_name= "test"
  fold_var = 1
  kf = KFold(n_splits = 5)
  my_optimizer= tf.keras.optimizers.Adam(learning_rate=LR)

  loss = np.zeros(5)

  for t_idx,v_idx in kf.split(input,labels):

    X_t,X_v = input[t_idx],input[v_idx]
    y_t,y_v = labels[t_idx],labels[v_idx]

    model = build_network()
    
    
    model.compile(optimizer=my_optimizer, loss='mean_squared_error')

    # CREATE CALLBACKS
    checkpoint = tf.keras.callbacks.ModelCheckpoint(f"{save_dir}{model_name}{fold_var}", 
              monitor='val_loss', 
              save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    

    # There can be other callbacks, but just showing one because it involves the model name
    # This saves the best model
    # FIT THE MODEL
    history = model.fit(X_t,y_t,
            epochs=EPOCHS,
            callbacks=callbacks_list,
            batch_size = BATCH,
            validation_data=(X_v,y_v))
    
    model.load_weights(f"{save_dir}{model_name}{fold_var}")
  
    loss[fold_var-1] = model.evaluate(X_v,y_v)

    fold_var += 1

    tf.keras.backend.clear_session()
  
  print(np.mean(loss))
  print(np.std(loss))






  # split into validation and test sets as you like

  # b)
  #  compile model using Adam optimizer (with learning rate of your choice) and MSE loss.

  # c)
  # fit model (use EPOCH for epoch parameter and BATCH for batch_size parameter)

  # d)
  # save model


  # part 3 predict


In [ ]:
train()
    



Epoch 1/3
